### Try 1

In [32]:
import pyrealsense2 as rs
import cv2
import datetime 
import numpy as np
import matplotlib.pyplot as plt

xaxis = 200
yaxis = 20

pipe = rs.pipeline()
cfg = rs.config()

cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipe.start(cfg)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

while True:
    frame = pipe.wait_for_frames()

        # Align the frames
    aligned_frames = align.process(frame)

    #depth_frame = frames.get_depth_frame()
    #color_frame = frames.get_color_frame()

    # Get the aligned color and depth frames
    aligned_color_frame = aligned_frames.get_color_frame()
    aligned_depth_frame = aligned_frames.get_depth_frame()


    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

    cv2.drawMarker(color_image, (xaxis, yaxis), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

    # Put images in the code
        # Get depth at coordinates (xaxis, yaxis)
    depth_value = depth_frame.get_distance(xaxis, yaxis)
    depth_text = f"Depth: {depth_value:.2f} meters"
    cv2.putText(color_image, depth_text, (xaxis+10, yaxis-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)


    cv2.imshow('rgb', color_image)
    cv2.imshow('depth', depth_cm)

    if cv2.waitKey(1) == ord('q'):
        break
    #plt.imshow(gray_image)
    #plt.scatter(xaxis, yaxis, color='red', marker='x')  # Mark the point (20, 20) with a red 'x'
    #plt.axis('off')  # Optional, to turn off the axis labels
    #plt.show()

    # Get depth at coordinates (xaxis, yaxis)
    depth_value = depth_frame.get_distance(xaxis, yaxis)
    print(f"Depth at ({xaxis}, {yaxis}): {depth_value} meters")

pipe.stop()


Depth at (200, 20): 1.1370000839233398 meters
Depth at (200, 20): 1.1460000276565552 meters
Depth at (200, 20): 1.1350001096725464 meters
Depth at (200, 20): 1.1420000791549683 meters
Depth at (200, 20): 1.1460000276565552 meters
Depth at (200, 20): 1.1500000953674316 meters
Depth at (200, 20): 1.1400001049041748 meters
Depth at (200, 20): 1.1440000534057617 meters
Depth at (200, 20): 1.1460000276565552 meters
Depth at (200, 20): 1.1350001096725464 meters
Depth at (200, 20): 1.1440000534057617 meters
Depth at (200, 20): 1.15500009059906 meters
Depth at (200, 20): 1.125 meters
Depth at (200, 20): 1.1570000648498535 meters
Depth at (200, 20): 1.1230000257492065 meters
Depth at (200, 20): 1.1290000677108765 meters
Depth at (200, 20): 1.152000069618225 meters
Depth at (200, 20): 1.1460000276565552 meters
Depth at (200, 20): 1.1370000839233398 meters
Depth at (200, 20): 1.152000069618225 meters
Depth at (200, 20): 1.1460000276565552 meters
Depth at (200, 20): 1.1420000791549683 meters
Depth

KeyboardInterrupt: 

### LUCA

In [39]:
import pyrealsense2 as rs
import cv2
import numpy as np

xaxis = 200
yaxis = 20

# define the screen size
width = 640
height = 480

# init. the camera pipeline
pipeline = rs.pipeline()
# camera settings
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
# start the camera pipeline
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# it is a good practice to wait a few frames before the main stream
for _ in range(10):
    pipeline.wait_for_frames()

frame_count = 0

try:
    while True:
        frame_count += 1

        frames = pipeline.wait_for_frames()

        # Align the frames
        aligned_frames = align.process(frames)

        # Get the aligned color and depth frames
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        # Get BGR and Depth data, then create a depth colorized image
        color_image = np.asanyarray(aligned_color_frame.get_data())
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Draw a marker at (xaxis, yaxis) on the color image
        cv2.drawMarker(color_image, (xaxis, yaxis), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Get depth at coordinates (xaxis, yaxis)
        depth_value = aligned_depth_frame.get_distance(xaxis, yaxis)
        depth_text = f"Depth: {depth_value:.2f} meters"
        cv2.putText(color_image, depth_text, (xaxis + 10, yaxis - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Stack the color image and depth colormap
        show_dual_win = np.hstack((color_image, depth_colormap))

        save_data(frame_count, depth_colormap, depth_value)

        # Show the color image with depth information
        cv2.imshow('Dual Window', show_dual_win)

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            # Quit
            break

        if pressed_key == ord('s'):
            # Save image or perform other actions
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


KeyboardInterrupt: 

## Project 

### Imports

In [ ]:
import pyrealsense2 as rs
import cv2
import numpy as np



### Inittialize vars and settings

In [78]:
# Coordinates of the two points
x1, y1 = 200, 20
x2, y2 = 200, 200

# Define the screen size
width = 640
height = 480

# Initialize the camera pipeline
pipeline = rs.pipeline()

# Camera settings
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)

# Start the camera pipeline
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Wait for a few frames before the main stream
# This is to allow the camera time to adjust its brightness and other camera settings and stabilize
for _ in range(10):
    pipeline.wait_for_frames()

# Get depth scale for converting the depth values to meters
# Intrinsics are used to get the depth scale

profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

try:
    while True:
        frames = pipeline.wait_for_frames()

        # Align the frames
        aligned_frames = align.process(frames)

        # Get the aligned color and depth frames
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        # Get BGR and Depth data, then create a depth colorized image
        color_image = np.asanyarray(aligned_color_frame.get_data())
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Draw markers at the two points on the color image
        cv2.drawMarker(color_image, (x1, y1), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
        cv2.drawMarker(color_image, (x2, y2), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Get depth values at the two points
        depth_value1 = aligned_depth_frame.get_distance(x1, y1)
        depth_value2 = aligned_depth_frame.get_distance(x2, y2)

        # Convert from pixel coordinates to real world coordinates 
        point1 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x1, y1], depth_value1)
        point2 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x2, y2], depth_value2)

        # Calculate the euclidean distance between the two points with depth values as z coordinates
        diff = np.subtract(point1, point2)
        euclidean_distance = np.linalg.norm(diff)

        # Display the distance on the color image
        distance_text = f"Distance: {euclidean_distance:.2f} meters"
        cv2.putText(color_image, distance_text, (x1 + 10, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        # Trace a line between the two points
        cv2.line(color_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Stack the color image and depth colormap
        show_dual_win = np.hstack((color_image, depth_colormap))

        # Show the color image with depth information
        cv2.imshow('Dual Window', show_dual_win)

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            # Quit
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()



### Now detection of 2 points that are not hardcoded and have the same color and the euclidean distance between them

In [34]:
# Coordinates of the two points
x1, y1 = 200, 20
x2, y2 = 200, 200

# Define the screen size
width = 640
height = 480

# Initialize the camera pipeline
pipeline = rs.pipeline()

# Camera settings
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)

# Start the camera pipeline
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Wait for a few frames before the main stream
# This is to allow the camera time to adjust its brightness and other camera settings and stabilize
for _ in range(10):
    pipeline.wait_for_frames()

# Get depth scale for converting the depth values to meters
# Intrinsics are used to get the depth scale

profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

try:
    while True:
        frames = pipeline.wait_for_frames()

        # Align the frames
        aligned_frames = align.process(frames)

        # Get the aligned color and depth frames
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue
        

        # Get BGR and Depth data, then create a depth colorized image
        color_image = np.asanyarray(aligned_color_frame.get_data())
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Draw markers at the two points on the color image
        cv2.drawMarker(color_image, (x1, y1), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
        cv2.drawMarker(color_image, (x2, y2), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Get depth values at the two points
        depth_value1 = aligned_depth_frame.get_distance(x1, y1)
        depth_value2 = aligned_depth_frame.get_distance(x2, y2)

        # Convert from pixel coordinates to real world coordinates 
        point1 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x1, y1], depth_value1)
        point2 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x2, y2], depth_value2)

        # Calculate the euclidean distance between the two points with depth values as z coordinates
        diff = np.subtract(point1, point2)
        euclidean_distance = np.linalg.norm(diff)

        # Display the distance on the color image
        distance_text = f"Distance: {euclidean_distance:.2f} meters"
        cv2.putText(color_image, distance_text, (x1 + 10, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        # Trace a line between the two points
        cv2.line(color_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Stack the color image and depth colormap
        show_dual_win = np.hstack((color_image, depth_colormap))

        
        # Show the color image with depth information
        cv2.imshow('Dual Window', show_dual_win)

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            # Quit
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()



KeyboardInterrupt: 

## Day 2

### Code 1


In [10]:
# mean of last 10 frames

def stabilization_values(last_depth_values: list, depth_value: float) -> float:
    # Add the depth value to the list and remove the oldest one if the list has more than 10 items
    last_depth_values.append(depth_value)
    if len(last_depth_values) > 50:
        last_depth_values.pop(0)

    # Calculate the mean depth value
    mean_depth_value = sum(last_depth_values) / len(last_depth_values)

    return mean_depth_value, last_depth_values


#### Save data

In [35]:
def save_data(frame_count: int, color_mtx, depth_mtx) -> None:
    # Save frames every 100 frames
    if frame_count % 100 == 0:x
        np.save(f'color_{frame_count}.npy', color_mtx)
        np.save(f'depth_{frame_count}.npy', depth_mtx)

IndentationError: unexpected indent (2369773669.py, line 4)

In [9]:
import pyrealsense2 as rs
import cv2
import numpy as np
import pickle
import time

xaxis = 320
yaxis = 240
width = 848
height = 480

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)
last_frame_values_depth = []
display_point = None
display_text = ""

frame_count = 0

# Callback function for mouse click
# TODO: correct the mean depth value calculation when the mouse is clicked
def mouse_click(event, x, y, flags, param):
    global display_point, display_text, last_frame_values_depth, mean_depth_value
    if event == cv2.EVENT_LBUTTONDOWN:
        for i in range(2):
            depth_value = aligned_depth_frame.get_distance(x, y) 
            display_point = (x, y)
            display_text = f"Distance: {depth_value:.2f} meters"
            time.sleep(1)

cv2.namedWindow('dual_win')
cv2.setMouseCallback('dual_win', mouse_click)

try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        color_mtx = np.asanyarray(aligned_color_frame.get_data())
        depth_mtx = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_mtx, alpha=0.03), cv2.COLORMAP_JET)

        frame_count += 1

        #cv2.drawMarker(color_mtx, (xaxis, yaxis), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
        show_dual_win = np.hstack((color_mtx,depth_colormap))
        
        pressed_key = cv2.waitKey(1) & 0xFF
        
        if pressed_key == ord('q'):
            break

        # Display the cross marker and distance at the last clicked location
        if display_point is not None:
            cv2.drawMarker(show_dual_win, display_point, (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
            cv2.putText(show_dual_win, display_text, (display_point[0] + 10, display_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
        mean_depth_value, last_frame_values_depth = stabilization_values(last_frame_values_depth, depth_value)

        cv2.imshow('dual_win', show_dual_win)

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


NameError: name 'stabilization_values' is not defined

#### Load data

In [104]:
import numpy as np
import cv2
import os


frame_count = len(os.listdir()) // 2  # Divide by 2 because we have color and depth frames
xaxis = 320
yaxis = 240

try:
    for i in range(100, 500, 100):  # multiply frame_count by 100 because we saved frames every 100 frames
        color_frame = np.load(f'color_{i}.npy')
        depth_frame = np.load(f'depth_{i}.npy')

        depth_value = depth_frame[yaxis, xaxis]

        # Display the distance on the color image
        distance_text = f"Distance: {depth_value:.2f} meters"
        cv2.putText(color_frame, distance_text, (xaxis + 10, yaxis - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv2.imshow('color_frame', color_frame)

        if cv2.waitKey(66) & 0xFF == ord('q'):  # Delay of 66 ms for approx 15 fps
            break
finally:
    cv2.destroyAllWindows()


### YOLO Model

In [1]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load COCO class names
with open("coco.names", "r") as f:
    class_names = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
unconnected_out_layers = net.getUnconnectedOutLayers()

if len(unconnected_out_layers.shape) == 2:  # Shape (n, 1) - OpenCV 3.x
    output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
else:  # Shape (n,) - OpenCV 4.x
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

# Load the video
cap = cv2.VideoCapture("video.mp4")

try:
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break

        height, width, channels = frame.shape

        

        # Detect objects
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Show information on screen
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # We use NMS function in opencv to perform Non-maximum Suppression
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # We draw bounding boxes and labels of detection
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(class_ids[i])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                label = f"{class_names[class_ids[i]]}: {confidences[i]:.2f}"  # use class_ids[i] and confidences[i]
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


        cv2.imshow("Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()


In [36]:
import pyrealsense2 as rs
import cv2
import numpy as np
import time

xaxis = 320
yaxis = 240
width = 848
height = 480

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)
last_frame_values_depth = []
display_point = None
display_text = ""

frame_count = 0

# Callback function for mouse click
# TODO: correct the mean depth value calculation when the mouse is clicked
def mouse_click(event, x, y, flags, param):
    global display_point, display_text, last_frame_values_depth, mean_depth_value
    if event == cv2.EVENT_LBUTTONDOWN:
        for i in range(2):
            depth_value = aligned_depth_frame.get_distance(x, y)
            display_point = (x, y)
            display_text = f"Distance: {depth_value:.2f} meters"
            time.sleep(1)


cv2.namedWindow('dual_win')
cv2.setMouseCallback('dual_win', mouse_click)

# Load YOLO
net = cv2.dnn.readNet("models/yolov3.weights", "models/yolov3.cfg")

# Load COCO class names
with open("coco.names", "r") as f:
    class_names = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
unconnected_out_layers = net.getUnconnectedOutLayers()

if len(unconnected_out_layers.shape) == 2:  # Shape (n, 1) - OpenCV 3.x
    output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
else:  # Shape (n,) - OpenCV 4.x
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

# Define video writer parameters
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_file = cv2.VideoWriter('yolo_try5.mp4', fourcc, 15.0, (2 * width, height))

try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        color_mtx = np.asanyarray(aligned_color_frame.get_data())
        depth_mtx = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_mtx, alpha=0.03), cv2.COLORMAP_JET)

        frame_count += 1

        # Detect objects using YOLO
        blob = cv2.dnn.blobFromImage(color_mtx, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Show information on screen
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Perform non-maximum suppression
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Draw bounding boxes and labels
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = f"{class_names[class_ids[i]]}: {confidences[i]:.2f}"
                cv2.rectangle(color_mtx, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(color_mtx, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        show_dual_win = np.hstack((color_mtx, depth_colormap))

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            break

        # Display the cross marker and distance at the last clicked location
        if display_point is not None:
            cv2.drawMarker(show_dual_win, display_point, (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
            cv2.putText(show_dual_win, display_text, (display_point[0] + 10, display_point[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv2.imshow('dual_win', show_dual_win)

        # Save the frame to the video file
        output_file.write(show_dual_win)

finally:
    pipeline.stop()
    output_file.release()
    cv2.destroyAllWindows()


### Detection of red rectangles


In [35]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Video frame dimensions
width = 640
height = 480

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

for _ in range(10):
    pipeline.wait_for_frames()

# Get depth scale for converting the depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Color range for red rectangle detection (in HSV color space)
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([180, 255, 255])

# Lists for tracking the depth values
last_depth_values1 = []
last_depth_values2 = []
last_depth_values_between = []

cv2.namedWindow('Color Image')
# Define video writer parameters
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_file = cv2.VideoWriter('rectangles_4.mp4', fourcc, 30.0, (width, height))


# Check if the output video file is open
if not output_file.isOpened():
    print("Error: Unable to open output video file")
    exit()
# Main loop
while True:
    # Wait for a coherent pair of frames: color and depth
    frames = pipeline.wait_for_frames()
    # Align the frames
    aligned_frames = align.process(frames)

    # Get the aligned color and depth frames
    color_frame = aligned_frames.get_color_frame()
    depth_frame = aligned_frames.get_depth_frame()

    if not color_frame or not depth_frame:
        continue

    # Convert color image to numpy array
    color_image = np.asanyarray(color_frame.get_data())

    # Convert color image to HSV color space
    hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)

    # Create a mask for red objects
    mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
    mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    mask_red_combined = cv2.bitwise_or(mask_red, mask_red2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask_red_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest and second largest red rectangles
    largest_rectangle = None
    second_largest_rectangle = None
    max_area = 0
    second_max_area = 0

    for contour in contours:
        # Find the minimum area rectangle
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        # Calculate the area of the rectangle
        area = cv2.contourArea(box)

        # Update the largest and second largest rectangles if the area is larger
        if area > max_area:
            second_largest_rectangle = largest_rectangle
            second_max_area = max_area
            max_area = area
            largest_rectangle = box
        elif area > second_max_area:
            second_max_area = area
            second_largest_rectangle = box

    if largest_rectangle is not None:
        # Draw the largest rectangle and its corners on the color image
        cv2.drawContours(color_image, [largest_rectangle], 0, (0, 0, 255), 2)
        for corner in largest_rectangle:
            x, y = corner
            cv2.circle(color_image, (x, y), 5, (0, 0, 255), -1)

        # Find the middle point of the rectangle
        rect_center = np.mean(largest_rectangle, axis=0)
        rect_center_x, rect_center_y = rect_center.astype(int)
        cv2.circle(color_image, (rect_center_x, rect_center_y), 5, (0, 255, 0), -1)

        # Get the depth value at the middle point
        depth_value1 = depth_frame.get_distance(rect_center_x, rect_center_y)

        # Convert from pixel coordinates to real-world coordinates
        depth_intrinsics = depth_frame.profile.as_video_stream_profile().intrinsics
        point1 = np.array(rs.rs2_deproject_pixel_to_point(depth_intrinsics, [rect_center_x, rect_center_y], depth_value1))

        # Stabilize the depth value
        mean_depth_value1, last_depth_values1 = stabilization_values(last_depth_values1, depth_value1)
        
        # Display the stabilized distance on the color image
        text = f"Stabilized Distance 1: {mean_depth_value1:.2f} meters"
        cv2.putText(color_image, text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    if second_largest_rectangle is not None:
        # Draw the second largest rectangle and its corners on the color image
        cv2.drawContours(color_image, [second_largest_rectangle], 0, (0, 255, 0), 2)
        for corner in second_largest_rectangle:
            x, y = corner
            cv2.circle(color_image, (x, y), 5, (0, 255, 0), -1)

        # Find the middle point of the second rectangle
        second_rect_center = np.mean(second_largest_rectangle, axis=0)
        second_rect_center_x, second_rect_center_y = second_rect_center.astype(int)
        cv2.circle(color_image, (second_rect_center_x, second_rect_center_y), 5, (255, 0, 0), -1)

        # Get the depth value at the middle point
        depth_value2 = depth_frame.get_distance(second_rect_center_x, second_rect_center_y)

        # Convert from pixel coordinates to real-world coordinates
        point2 = np.array(rs.rs2_deproject_pixel_to_point(depth_intrinsics, [second_rect_center_x, second_rect_center_y], depth_value2))

    # Stabilize the depth value
        mean_depth_value2, last_depth_values2 = stabilization_values(last_depth_values2, depth_value2)
        
        # Display the stabilized distance on the color image
        text = f"Stabilized Distance 2: {mean_depth_value2 :.2f} meters"
        cv2.putText(color_image, text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


        # Calculate the Euclidean distance between the two middle points
        distance = np.linalg.norm(point2 - point1)

        # Display the Euclidean distance between the two middle points on the color image
        distance_text = f"Distance between points: {distance:.2f} meters"
        cv2.putText(color_image, distance_text, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Draw a line between the two middle points
        cv2.line(color_image, (rect_center_x, rect_center_y), (second_rect_center_x, second_rect_center_y), (255, 0, 0), 2)

    # Display the color image
    cv2.imshow("Color Image", color_image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    output_file.write(color_image)

pipeline.stop()
output_file.release()
cv2.destroyAllWindows()

C:\Users\leo_b\AppData\Local\Temp\ipykernel_17836\3568115185.py:89: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


### Tic tac toe

In [51]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Configure the streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)


# The game state
game_state = np.zeros((3, 3), dtype=int)

# Mapping from shape to player
shape_to_player = {'triangle': 1, 'circle': -1}

# A function to update the game state based on a shape's center
def update_game_state(shape, center):
    player = shape_to_player[shape]
    i = int(center[1] // cell_height)
    j = int(center[0] // cell_width)
    game_state[i, j] = player

# A function to check if there's a win
def check_win():
    # Check rows and columns
    for i in range(3):
        if abs(sum(game_state[i, :])) == 3 or abs(sum(game_state[:, i])) == 3:
            return True
    # Check diagonals
    if abs(sum(game_state[i, i] for i in range(3))) == 3 or abs(sum(game_state[i, 2 - i] for i in range(3))) == 3:
        return True
    return False

cv2.namedWindow('Color Image')
# Define video writer parameters
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_file = cv2.VideoWriter('tictactoe.mp4', fourcc, 30.0, (width, height))


# Check if the output video file is open
if not output_file.isOpened():
    print("Error: Unable to open output video file")
    exit()


try:
    while True:
        # Wait for a new set of frames: depth and color
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

                # Calculate the size of each grid cell
        height, width, _ = color_image.shape
        cell_width = width // 3
        cell_height = height // 3

        # Draw vertical lines
        for i in range(1, 3):
            cv2.line(color_image, (i * cell_width, 0), (i * cell_width, height), (255, 255, 255), 2)

        # Draw horizontal lines
        for i in range(1, 3):
            cv2.line(color_image, (0, i * cell_height), (width, i * cell_height), (255, 255, 255), 2)
        

        # Convert the image to grayscale
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur for smoothing
        blur = cv2.GaussianBlur(gray, (5, 5), 0)

        # Apply edge detection
        edges = cv2.Canny(blur, 50, 150, apertureSize=3)

        # Find contours in the image
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        shape_centers = {'triangle': [], 'circle': []}

        for cnt in contours:
            # Get contour area
            area = cv2.contourArea(cnt)

            # Ignore small contours
            if area < 400:
                continue

            # Find the perimeter of contour
            perimeter = cv2.arcLength(cnt, True)

            # Get approximate polygon
            approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)

            if len(approx) == 3:
                x, y = np.average(approx, axis=0)[0]
                shape_centers['triangle'].append((x, y))
                cv2.drawContours(color_image, [approx], 0, (0, 255, 0), 5)
                cv2.putText(color_image, "Triangle", (int(x), int(y) - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                # Fit an ellipse if contour has more than 5 points
                if len(approx) > 5:
                    ellipse = cv2.fitEllipse(approx)
                    x, y = np.average(approx, axis=0)[0]
                    shape_centers['circle'].append((x, y))
                    cv2.ellipse(color_image, ellipse, (0, 0, 255), 2)
                    cv2.putText(color_image, "Circle", (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # In your loop where you detect shapes, update the game state
        for shape in shape_centers:
            for center in shape_centers[shape]:
                update_game_state(shape, center)
        
        if check_win():
            cv2.putText(color_image, "WINNER!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)    

        # Show image
        cv2.imshow('Shapes detection', color_image)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        output_file.write(color_image)


finally:
    # Stop streaming
    pipeline.stop()
    output_file.release()
    cv2.destroyAllWindows()


### Pitagoras


In [41]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Video frame dimensions
width = 640
height = 480

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

for _ in range(10):
    pipeline.wait_for_frames()

# Get depth scale for converting the depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Color range for red rectangle detection (in HSV color space)
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([180, 255, 255])

# Lists for tracking the depth values
last_depth_values1 = []
last_depth_values2 = []
last_depth_values3 = []  # New list for third point
last_depth_values_between = []

# Find the largest, second largest and third largest red rectangles
largest_rectangle = None
second_largest_rectangle = None
third_largest_rectangle = None  # New variable for third rectangle
max_area = 0
second_max_area = 0
third_max_area = 0  # New variable for third area

cv2.namedWindow('Color Image')
# Define video writer parameters
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_file = cv2.VideoWriter('rectangles_area.mp4', fourcc, 30.0, (width, height))


# Check if the output video file is open
if not output_file.isOpened():
    print("Error: Unable to open output video file")
    exit()
# Main loop
while True:
    # Wait for a coherent pair of frames: color and depth
    frames = pipeline.wait_for_frames()
    # Align the frames
    aligned_frames = align.process(frames)

    # Get the aligned color and depth frames
    color_frame = aligned_frames.get_color_frame()
    depth_frame = aligned_frames.get_depth_frame()

    if not color_frame or not depth_frame:
        continue

    # Convert color image to numpy array
    color_image = np.asanyarray(color_frame.get_data())

    # Convert color image to HSV color space
    hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)

    # Create a mask for red objects
    mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
    mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    mask_red_combined = cv2.bitwise_or(mask_red, mask_red2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask_red_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest and second largest red rectangles
    largest_rectangle = None
    second_largest_rectangle = None
    max_area = 0
    second_max_area = 0

    for contour in contours:
        # Find the minimum area rectangle
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        # Calculate the area of the rectangle
        area = cv2.contourArea(box)


        # Update the largest, second largest and third largest rectangles if the area is larger
        if area > max_area:
            third_largest_rectangle = second_largest_rectangle  # New line
            third_max_area = second_max_area  # New line
            second_largest_rectangle = largest_rectangle
            second_max_area = max_area
            max_area = area
            largest_rectangle = box
        elif area > second_max_area:
            third_max_area = second_max_area  # New line
            third_largest_rectangle = second_largest_rectangle  # New line
            second_max_area = area
            second_largest_rectangle = box
        elif area > third_max_area:  # New condition for third rectangle
            third_max_area = area
            third_largest_rectangle = box

    if largest_rectangle is not None:
        # Draw the largest rectangle and its corners on the color image
        cv2.drawContours(color_image, [largest_rectangle], 0, (0, 0, 255), 2)
        for corner in largest_rectangle:
            x, y = corner
            cv2.circle(color_image, (x, y), 5, (0, 0, 255), -1)

        # Find the middle point of the rectangle
        rect_center = np.mean(largest_rectangle, axis=0)
        rect_center_x, rect_center_y = rect_center.astype(int)
        cv2.circle(color_image, (rect_center_x, rect_center_y), 5, (0, 255, 0), -1)

        # Get the depth value at the middle point
        depth_value1 = depth_frame.get_distance(rect_center_x, rect_center_y)

        # Convert from pixel coordinates to real-world coordinates
        depth_intrinsics = depth_frame.profile.as_video_stream_profile().intrinsics
        point1 = np.array(rs.rs2_deproject_pixel_to_point(depth_intrinsics, [rect_center_x, rect_center_y], depth_value1))

        # Stabilize the depth value
        mean_depth_value1, last_depth_values1 = stabilization_values(last_depth_values1, depth_value1)
        
        # Display the stabilized distance on the color image
        text = f"Stabilized Distance 1: {mean_depth_value1:.2f} meters"
        cv2.putText(color_image, text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    if second_largest_rectangle is not None:
        # Draw the second largest rectangle and its corners on the color image
        cv2.drawContours(color_image, [second_largest_rectangle], 0, (0, 255, 0), 2)
        for corner in second_largest_rectangle:
            x, y = corner
            cv2.circle(color_image, (x, y), 5, (0, 255, 0), -1)

        # Find the middle point of the second rectangle
        second_rect_center = np.mean(second_largest_rectangle, axis=0)
        second_rect_center_x, second_rect_center_y = second_rect_center.astype(int)
        cv2.circle(color_image, (second_rect_center_x, second_rect_center_y), 5, (255, 0, 0), -1)

        # Get the depth value at the middle point
        depth_value2 = depth_frame.get_distance(second_rect_center_x, second_rect_center_y)

        # Convert from pixel coordinates to real-world coordinates
        point2 = np.array(rs.rs2_deproject_pixel_to_point(depth_intrinsics, [second_rect_center_x, second_rect_center_y], depth_value2))

    # Stabilize the depth value
        mean_depth_value2, last_depth_values2 = stabilization_values(last_depth_values2, depth_value2)
        
        # Display the stabilized distance on the color image
        text = f"Stabilized Distance 2: {mean_depth_value2 :.2f} meters"
        cv2.putText(color_image, text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


        # Calculate the Euclidean distance between the two middle points
        distance = np.linalg.norm(point2 - point1)

        # Display the Euclidean distance between the two middle points on the color image
        distance_text = f"Distance between points: {distance:.2f} meters"
        cv2.putText(color_image, distance_text, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Draw a line between the two middle points
        cv2.line(color_image, (rect_center_x, rect_center_y), (second_rect_center_x, second_rect_center_y), (255, 0, 0), 2)

        if third_largest_rectangle is not None:  # New block for third rectangle
            cv2.drawContours(color_image, [third_largest_rectangle], 0, (255, 0, 255), 2)
            for corner in third_largest_rectangle:
                x, y = corner
                cv2.circle(color_image, (x, y), 5, (255, 0, 255), -1)
            third_rect_center = np.mean(third_largest_rectangle, axis=0)
            third_rect_center_x, third_rect_center_y = third_rect_center.astype(int)
            cv2.circle(color_image, (third_rect_center_x, third_rect_center_y), 5, (255, 255, 0), -1)
            depth_value3 = depth_frame.get_distance(third_rect_center_x, third_rect_center_y)
            point3 = np.array(rs.rs2_deproject_pixel_to_point(depth_intrinsics, [third_rect_center_x, third_rect_center_y], depth_value3))
            mean_depth_value3, last_depth_values3 = stabilization_values(last_depth_values3, depth_value3)
            text = f"Stabilized Distance 3: {mean_depth_value3:.2f} meters"
            cv2.putText(color_image, text, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            distance3 = np.linalg.norm(point3 - point1)
            distance_text3 = f"Distance 3: {distance3:.2f} meters"
            cv2.putText(color_image, distance_text3, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.line(color_image, (rect_center_x, rect_center_y), (third_rect_center_x, third_rect_center_y), (255, 0, 0), 2)

            # Calculate and display the distances between all three points
            distance_12 = np.linalg.norm(point1 - point2)
            distance_23 = np.linalg.norm(point2 - point3)
            distance_31 = np.linalg.norm(point3 - point1)
            distance_text_12 = f"Distance between points 1 and 2: {distance_12:.2f} meters"
            distance_text_23 = f"Distance between points 2 and 3: {distance_23:.2f} meters"
            distance_text_31 = f"Distance between points 3 and 1: {distance_31:.2f} meters"
            cv2.putText(color_image, distance_text_12, (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(color_image, distance_text_23, (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(color_image, distance_text_31, (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.line(color_image, (second_rect_center_x, second_rect_center_y), (third_rect_center_x, third_rect_center_y), (255, 0, 0), 2)

                # Calculate the semi-perimeter
            s = (distance_12 + distance_23 + distance_31) / 2

            # Calculate the area
            area = np.sqrt(s * (s - distance_12) * (s - distance_23) * (s - distance_31))

            # Display the area on the color image
            area_text = f"Area between points: {area:.2f} square meters"
            cv2.putText(color_image, area_text, (10, 270), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


    # Display the color image
    cv2.imshow("Color Image", color_image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    output_file.write(color_image)

pipeline.stop()
output_file.release()
cv2.destroyAllWindows()

C:\Users\leo_b\AppData\Local\Temp\ipykernel_17836\949684978.py:98: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
